Hasta ahora, solo hemos utilizado la API de alto nivel de TensorFlow, Keras, pero ya nos ha llegado bastante lejos: construimos varias arquitecturas de redes neuronales, incluyendo redes de regresión y clasificación, redes Wide & Deep, y redes de autonormalización, utilizando todo tipo de técnicas, como la normalización por lotes, la abandono y los horarios de tasa de aprendizaje. De hecho, el 95 % de los casos de uso que encontrará no requerirán nada más que Keras (y tf.data; véase el capítulo 13). Pero ahora es el momento de profundizar en TensorFlow y echar un vistazo a su API de Python de nivel inferior. Esto será útil cuando necesite un control adicional para escribir funciones de pérdida personalizadas, métricas personalizadas, capas, modelos, inicializadores, regularizadores, restricciones de peso y más. Es posible que incluso necesite controlar completamente el bucle de entrenamiento en sí; por ejemplo, aplicar transformaciones o restricciones especiales a los gradientes (más allá de solo recortarlos) o usar múltiples optimizadores para diferentes partes de la red. Cubriremos todos estos casos en este capítulo, y también veremos cómo puede impulsar sus modelos personalizados y algoritmos de entrenamiento utilizando la función de generación automática de gráficos de TensorFlow. Pero primero, hagamos un recorrido rápido por TensorFlow.

## Un recorrido rápido por TensorFlow

Como saben, TensorFlow es una poderosa biblioteca para la computación numérica, particularmente adecuada y afinada para el aprendizaje automático a gran escala (pero puede usarla para cualquier otra cosa que requiera cálculos pesados). Fue desarrollado por el equipo de Google Brain y impulsa muchos de los servicios a gran escala de Google, como Google Cloud Speech, Google Photos y Google Search. Fue de código abierto en noviembre de 2015, y ahora es la biblioteca de aprendizaje profundo más utilizada en la industria: 1 innumerables proyectos utilizan TensorFlow para todo tipo de tareas de aprendizaje automático, como la clasificación de imágenes, el procesamiento del lenguaje natural, los sistemas de recomendación y la previsión de series temporales.

Entonces, ¿qué ofrece TensorFlow? Aquí hay un resumen:

* Su núcleo es muy similar a NumPy, pero con soporte para GPU.

- Es compatible con la computación distribuida (a través de múltiples dispositivos y servidores).

* Incluye un tipo de compilador justo a tiempo (JIT) que le permite optimizar los cálculos para la velocidad y el uso de la memoria. Funciona extrayendo el gráfico de cálculo de una función de Python, optimicándolo (por ejemplo, podando los nodos no utilizados) y ejecutándolo de manera eficiente (por ejemplo, ejecutando automáticamente operaciones independientes en paralelo).

- Los gráficos de computación se pueden exportar a un formato portátil, por lo que puede entrenar un modelo de TensorFlow en un entorno (por ejemplo, usando Python en Linux) y ejecutarlo en otro (por ejemplo, usando Java en un dispositivo Android).

* Implementa el autodiff en modo inverso (ver Capítulo 10 y Apéndice B) y proporciona algunos optimizadores excelentes, como RMSProp y Nadam (ver Capítulo 11), para que pueda minimizar fácilmente todo tipo de funciones de pérdida.

TensorFlow ofrece muchas más características construidas sobre estas características principales: la más importante es, por supuesto, Keras, ⁠2, pero también tiene operaciones de carga y preprocesamiento de datos (tf.data, tf.io, etc.), operaciones de procesamiento de imágenes (tf.image), operaciones de procesamiento de señales (tf.signal) y más (consulte la Figura 12-1 para obtener una descripción general de la API de Python de TensorFlow).


#### TIP

Cubriremos muchos de los paquetes y funciones de la API de TensorFlow, pero es imposible cubrirlos todos, por lo que realmente deberías tomarte un tiempo para navegar por la API; encontrarás que es bastante rica y está bien documentada.

#### ----------------------------------------------------------------------------------------------


En el nivel más bajo, cada operación de TensorFlow (op para abreviar) se implementa utilizando código C++ altamente eficiente.⁠ Muchas operaciones tienen múltiples implementaciones llamadas núcleos: cada núcleo está dedicado a un tipo de dispositivo específico, como CPU, GPU o incluso TPU (unidades de procesamiento de tensores). Como sabrás, las GPU pueden acelerar drásticamente los cálculos dividiéndolos en muchos trozos más pequeños y ejecutándolos en paralelo a través de muchos subprocesos de GPU. Los TPU son aún más rápidos: son chips ASIC personalizados construidos específicamente para operaciones de aprendizaje profundo⁠4 (discutiremos cómo usar TensorFlow con GPU o TPU en el capítulo 19).


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1201.png)

(_Figura 12-1. API de Python de TensorFlow_)


La arquitectura de TensorFlow se muestra en la Figura 12-2. La mayoría de las veces, su código utilizará las API de alto nivel (especialmente Keras y tf.data), pero cuando necesite más flexibilidad, utilizará la API de Python de nivel inferior, manejando los tensores directamente. En cualquier caso, el motor de ejecución de TensorFlow se encargará de ejecutar las operaciones de manera eficiente, incluso a través de múltiples dispositivos y máquinas, si se lo dice.

TensorFlow se ejecuta no solo en Windows, Linux y macOS, sino también en dispositivos móviles (usando TensorFlow Lite), incluidos iOS y Android (ver Capítulo 19). Tenga en cuenta que las API para otros lenguajes también están disponibles, si no desea usar la API de Python: hay API de C++, Java y Swift. Incluso hay una implementación de JavaScript llamada TensorFlow.js que permite ejecutar sus modelos directamente en su navegador.


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1202.png)


Hay más en TensorFlow que en la biblioteca. TensorFlow está en el centro de un extenso ecosistema de bibliotecas. En primer lugar, está TensorBoard para la visualización (ver Capítulo 10). A continuación, estáTensorFlow Extended (TFX), que es un conjunto de bibliotecas creadas por Google para producir proyectos de TensorFlow: incluye herramientas para la validación de datos, el preprocesamiento, el análisis de modelos y el servicio (con TF Serving; véase el capítulo 19). El TensorFlow Hub de Google proporciona una forma de descargar y reutilizar fácilmente las redes neuronales preentrenadas. También puedes obtener muchas arquitecturas de redes neuronales, algunas de ellas preentrenadas, en el jardín de modelos de TensorFlow. Echa un vistazo a los recursos de TensorFlow y https://github.com/jtoy/awesome-tensorflow para ver más proyectos basados en TensorFlow. Encontrarás cientos de proyectos de TensorFlow en GitHub, por lo que a menudo es fácil encontrar el código existente para lo que sea que estés tratando de hacer.


#### TIP

Cada vez se publican más documentos de aprendizaje automático junto con sus implementaciones, y a veces incluso con modelos preentrenados. Echa un vistazo a https://paperswithcode.com para encontrarlos fácilmente.

#### ----------------------------------------------------------------------------------------------


Por último, pero no menos importante, TensorFlow tiene un equipo dedicado de desarrolladores apasionados y útiles, así como una gran comunidad que contribuye a mejorarlo. Para hacer preguntas técnicas, debe usar https://stackoverflow.com y etiquetar su pregunta con tensorflow y python. Puedes presentar errores y solicitudes de funciones a través de GitHub. Para discusiones generales, únete al Foro de TensorFlow.

Vale, ¡es hora de empezar a programar!


## Usando Tensorflow como Numpy

La API de TensorFlow gira en torno a tensores, que fluyen de una operación a otra, de ahí el nombre TensorFlow. Un tensor es muy similar a un `ndarray` de NumPy: normalmente es una matriz multidimensional, pero también puede contener un escalar (un valor simple, como `42`). Estos tensores serán importantes cuando creemos funciones de costos personalizadas, métricas personalizadas, capas personalizadas y más, así que veamos cómo crearlos y manipularlos.

### Tensores y operaciones

Puedes crear un tensor con `tf.constant()`. Por ejemplo, aquí hay un tensor que representa una matriz con dos filas y tres columnas de flotadores:

In [2]:
import tensorflow as tf

t = tf.constant([[1., 2., 3.], [4., 5., 6.]])  # matrix
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

Al igual que un `ndarray`, un `tf.Tensor` tiene una forma y un tipo de datos (`dtype`):

In [3]:
t.shape

TensorShape([2, 3])

In [4]:
t.dtype

tf.float32

La indexación funciona de manera muy similar a NumPy:

In [5]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [6]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

Lo más importante es que todo tipo de operaciones de tensor están disponibles:

In [7]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [8]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [9]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

Tenga en cuenta que escribir `t + 10` equivale a llamar a `tf.add(t, 10)` (de hecho, Python llama al método mágico `t.__add__(10)`, que simplemente llama a `tf.add(t, 10)`). También se admiten otros operadores, como `-` y `*`. El operador `@` se agregó en Python 3.5, para la multiplicación de matrices: equivale a llamar a la función `tf.matmul()`.

#### NOTA

Muchas funciones y clases tienen alias. Por ejemplo, `tf.add()` y `tf.math.add()` son la misma función. Esto permite que TensorFlow tenga nombres concisos para las operaciones más comunes⁠ y al mismo tiempo preserve los paquetes bien organizados.

#### ---------------------------------------------------------------------------------------------------


Un tensor también puede contener un valor escalar. En este caso, la forma está vacía:

In [10]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

#### NOTA

La API de Keras tiene su propia API de bajo nivel, ubicada en `tf.keras.backend`. Este paquete normalmente se importa como `K`, por razones de concisión. Solía incluir funciones como `K.square()`, `K.exp()` y `K.sqrt()`, que puede encontrar en el código existente: esto era útil para escribir código portátil cuando Keras admitía múltiples backends, pero ahora Para saber que Keras es solo para TensorFlow, debes llamar directamente a la API de bajo nivel de TensorFlow (por ejemplo, `tf.square()` en lugar de `K.square()`). Técnicamente, `K.square()` y sus amigos todavía están ahí por compatibilidad con versiones anteriores, pero la documentación del paquete `tf.keras.backend` solo enumera un puñado de funciones de utilidad, como `clear_session()` (mencionada en el Capítulo 10).

#### -----------------------------------------------------------------------------------


Encontrará todas las operaciones matemáticas básicas que necesita (`tf.add()`, `tf.multiply()`, `tf.square()`, `tf.exp()`, `tf.sqrt()`, etc.) y la mayoría de las operaciones que pueda buscar en NumPy (por ejemplo, `tf.reshape()`, `tf.squeeze()`, `tf.tile()`). Algunas funciones tienen un nombre diferente al de NumPy; por ejemplo, `tf.reduce_mean()`, `tf.reduce_sum()`, `tf.reduce_max()` y `tf.math.log()` son el equivalente de `np.mean()`, `np.sum()`, `np.max()` y `np.log()`. Cuando el nombre difiere, suele haber una buena razón para ello. Por ejemplo, en TensorFlow debes escribir `tf.transpose(t)`; no puedes simplemente escribir `t.T` como en NumPy. La razón es que la función `tf.transpose()` no hace exactamente lo mismo que el atributo `T` de NumPy: en TensorFlow, se crea un nuevo tensor con su propia copia de los datos transpuestos, mientras que en NumPy, `t.T` es solo una vista transpuesta. sobre los mismos datos. De manera similar, la operación `tf.reduce_sum()` se denomina de esta manera porque su núcleo de GPU (es decir, la implementación de GPU) utiliza un algoritmo de reducción que no garantiza el orden en el que se agregan los elementos: debido a que los flotantes de 32 bits tienen una precisión limitada, el resultado puede cambiar ligeramente cada vez que llame a esta operación. Lo mismo ocurre con `tf.reduce_mean()` (pero, por supuesto, `tf.reduce_max()` es determinista).


### Tensores y Numpy

Los tensores juegan bien con NumPy: puedes crear un tensor a partir de una matriz de NumPy, y viceversa. Incluso puede aplicar operaciones de TensorFlow a matrices NumPy y operaciones NumPy a tensores:

In [11]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [12]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [13]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [14]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

#### ADVERTENCIA

Tenga en cuenta que NumPy utiliza una precisión de 64 bits de forma predeterminada, mientras que TensorFlow utiliza 32 bits. Esto se debe a que la precisión de 32 bits es generalmente más que suficiente para las redes neuronales, además de que funciona más rápido y utiliza menos RAM. Así que cuando crees un tensor a partir de una matriz NumPy, asegúrate de establecer `dtype=tf.float32`.

### Tipo de coversiones

Las conversiones de tipo pueden perjudicar significativamente el rendimiento, y pueden pasar fácilmente desapercibidas cuando se hacen automáticamente. Para evitar esto, TensorFlow no realiza ninguna conversión de tipo automáticamente: solo genera una excepción si intenta ejecutar una operación en tensores con tipos incompatibles. Por ejemplo, no se puede añadir un tensor flotante y un tensor entero, y ni siquiera se puede añadir un flotante de 32 bits y un flotante de 64 bits:


In [15]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [16]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

Esto puede resultar un poco molesto al principio, ¡pero recuerda que es por una buena causa! Y, por supuesto, puedes usar `tf.cast()` cuando realmente necesites convertir tipos:

In [17]:
t2 = tf.constant(40., dtype=tf.float64)

In [18]:
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

Los valores de `tf.Tensor` que hemos visto hasta ahora son inmutables: no podemos modificarlos. Esto significa que no podemos usar tensores regulares para implementar pesos en una red neuronal, ya que es necesario modificarlos mediante retropropagación. Además, es posible que también sea necesario cambiar otros parámetros con el tiempo (por ejemplo, un optimizador de impulso realiza un seguimiento de los gradientes pasados). Lo que necesitamos es una `tf.Variable`:

In [19]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

Un `tf.Variable` actúa de manera muy similar a un `tf.Tensor`: puedes realizar las mismas operaciones con él, también funciona muy bien con NumPy y es igual de exigente con los tipos. Pero también se puede modificar en el lugar usando el método `assign()` (o `assign_add()` o `assigs_sub()`, que incrementan o disminuyen la variable en el valor dado). También puede modificar celdas (o sectores) individuales, utilizando el método `assign()` de la celda (o sector) o utilizando los métodos `scatter_update()` o `scatter_nd_update()`:

In [20]:
v.assign(2 * v)           # v now equals [[2., 4., 6.], [8., 10., 12.]]
v[0, 1].assign(42)        # v now equals [[2., 42., 6.], [8., 10., 12.]]
v[:, 2].assign([0., 1.])  # v now equals [[2., 42., 0.], [8., 10., 1.]]
v.scatter_nd_update(      # v now equals [[100., 42., 0.], [8., 10., 200.]]
    indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

La asignación directa no funcionará:

In [21]:
v[1] = [7., 8., 9.]

TypeError: 'ResourceVariable' object does not support item assignment

#### NOTA

En la práctica, rara vez tendrás que crear variables manualmente; Keras proporciona un método `add_weight()` que se encargará de ello por usted, como verá. Además, los optimizadores generalmente actualizarán los parámetros del modelo directamente, por lo que rara vez necesitará actualizar las variables manualmente.

#### -------------------------------------------------------------------------------

### Otras estructuras de datos

TensorFlow es compatible con varias otras estructuras de datos, incluida la siguiente (consulte la sección "Otras estructuras de datos" en el cuaderno de este capítulo o en el Apéndice C para obtener más detalles):

* _Tensores dispersos_ (`tf.SparseTensor`)

    Representa eficientemente tensores que contienen principalmente ceros. El paquete `tf.sparse` contiene operaciones para tensores dispersos.

- _Matrices de tensores_ (`tf.TensorArray`)

    Son listas de tensores. Tienen una longitud fija por defecto, pero opcionalmente se pueden hacer extensibles. Todos los tensores que contienen deben tener la misma forma y tipo de datos.
    
* _Tensores irregulares_ (`tf.RaggedTensor`)

    Representa listas de tensores, todos del mismo rango y tipo de datos, pero con diferentes tamaños. Las dimensiones a lo largo de las cuales varían los tamaños de los tensores se denominan dimensiones irregulares. El paquete `tf.ragged` contiene operaciones para tensores irregulares.

- _Tensores de strings_

    Son tensores regulares de tipo `tf.string`. Estos representan cadenas de bytes, no cadenas Unicode, por lo que si crea un tensor de cadena usando una cadena Unicode (por ejemplo, una cadena normal de Python 3 como `"café"`), se codificará en UTF-8 automáticamente (por ejemplo, ` b"caf\xc3\xa9"`). Alternativamente, puede representar cadenas Unicode usando tensores de tipo `tf.int32`, donde cada elemento representa un punto de código Unicode (por ejemplo, `[99, 97, 102, 233]`). El paquete `tf.strings` (con una s) contiene operaciones para cadenas de bytes y cadenas Unicode (y para convertir una en otra). Es importante tener en cuenta que "tf.string" es atómico, lo que significa que su longitud no aparece en la forma del tensor. Una vez que lo convierte a un tensor Unicode (es decir, un tensor de tipo `tf.int32` que contiene puntos de código Unicode), la longitud aparece en la forma.
    
* _Conjuntos_

    Se representan como tensores regulares (o tensores dispersos). Por ejemplo, `tf.constant([[1, 2], [3, 4]])` representa los dos conjuntos {1, 2} y {3, 4}. De manera más general, cada conjunto está representado por un vector en el último eje del tensor. Puede manipular conjuntos utilizando operaciones del paquete `tf.sets`.

- _Colas_

    Almacenar tensores en varios pasos. TensorFlow ofrece varios tipos de colas: colas básicas de primera entrada, primera salida (`FIFOQueue`), además de colas que pueden priorizar algunos elementos (`PriorityQueue`), barajar sus elementos (`RandomShuffleQueue`) y lotes de elementos de diferentes formas mediante relleno (`PaddingFIFOQueue`). Estas clases están todas en el paquete thetf `tf.queue`.

Con tensores, operaciones, variables y varias estructuras de datos a su disposición, ¡ahora está listo para personalizar sus modelos y algoritmos de entrenamiento!

## Personalización de modelos y algoritmos de entrenamiento

Comenzará creando una función de pérdida personalizada, que es un caso de uso sencillo y común.

### Funciones de pérdida personalizadas

Supongamos que quieres entrenar un modelo de regresión, pero tu conjunto de entrenamiento es un poco ruidoso. Por supuesto, comienzas tratando de limpiar tu conjunto de datos eliminando o arreglando los valores atípicos, pero eso resulta ser insuficiente; el conjunto de datos sigue siendo ruidoso. ¿Qué función de pérdida deberías usar? El error cuadrado medio podría penalizar demasiado los errores grandes y hacer que su modelo sea impreciso. El error absoluto medio no penalizaría tanto a los valores atípicos, pero el entrenamiento podría tardar un tiempo en converger, y el modelo entrenado podría no ser muy preciso. Este es probablemente un buen momento para usar la pérdida de Huber (introducida en el capítulo 10) en lugar de la buena y vieja MSE. La pérdida de Huber está disponible en Keras (solo usa una instancia de la clase `tf.keras.losses.Huber`), pero finjamos que no está ahí. Para implementarlo, simplemente cree una función que tome las etiquetas y las predicciones del modelo como argumentos, y utilice las operaciones de TensorFlow para calcular un tensor que contenga todas las pérdidas (una por muestra):

In [22]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

#### ADVERTENCIA

Para un mejor rendimiento, debe utilizar una implementación vectorizada, como en este ejemplo. Además, si desea beneficiarse de las funciones de optimización de gráficos de TensorFlow, debe utilizar solo las operaciones de TensorFlow.

#### -----------------------------------------------------------------------------------

También es posible devolver la pérdida media en lugar de las pérdidas individuales de la muestra, pero esto no se recomienda, ya que hace imposible el uso de pesos de clase o pesos de muestra cuando los necesita (ver Capítulo 10).

Ahora puedes usar esta función de pérdida de Huber cuando compiles el modelo de Keras, y luego entrena tu modelo como de costumbre:

In [ ]:
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, [...])

¡Y eso es! Para cada lote durante el entrenamiento, Keras llamará a la función `huber_fn()` para calcular la pérdida, luego utilizará autodiff en modo inverso para calcular los gradientes de la pérdida con respecto a todos los parámetros del modelo y, finalmente, realizará un descenso de gradiente. paso (en este ejemplo usando un optimizador Nadam). Además, realizará un seguimiento de la pérdida total desde el comienzo de la época y mostrará la pérdida media.

Pero, ¿qué pasa con esta pérdida personalizada cuando guardas el modelo?


### Guardar y cargar modelos que contienen componentes personalizados

Guardar un modelo que contenga una función de pérdida personalizada funciona bien, pero cuando lo cargue, deberá proporcionar un diccionario que asigna el nombre de la función a la función real. De manera más general, cuando carga un modelo que contiene objetos personalizados, debe asignar los nombres a los objetos:

In [ ]:
model = tf.keras.models.load_model("my_model_with_a_custom_loss",
                                   custom_objects={"huber_fn": huber_fn})

#### PROPINA

Si decoras la función `huber_fn()` con `@keras.utils.reg⁠ister_keras_serializable()`, automáticamente estará disponible para la función `load_model()`: no es necesario incluirla en el diccionario `custom_objects` .

Con la implementación actual, cualquier error entre -1 y 1 se considera "pequeño". Pero, ¿y si quieres un umbral diferente? Una solución es crear una función que cree una función de pérdida configurada:

In [ ]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

Desafortunadamente, cuando guarde el modelo, el "umbral" no se guardará. Esto significa que tendrá que especificar el valor `umbral` al cargar el modelo (tenga en cuenta que el nombre a usar es `"huber_fn"`, que es el nombre de la función que le dio a Keras, no el nombre de la función que creó él):

In [ ]:
model = tf.keras.models.load_model(
    "my_model_with_a_custom_loss_threshold_2",
    custom_objects={"huber_fn": create_huber(2.0)}
)

Puedes resolver esto creando una subclase de la clase thetf `tf.keras.losses.Loss`, y luego implementando el método its get `get_config()`:

In [ ]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

Vamos a revisar este código:

* The constructor accepts `**kwargs` and passes them to the parent constructor, which handles standard hyperparameters: the `name` of the loss and the `reduction` algorithm to use to aggregate the individual instance losses. By default this is `"AUTO"`, which is equivalent to `"SUM_OVER_BATCH_SIZE"`: the loss will be the sum of the instance losses, weighted by the sample weights, if any, and divided by the batch size (not by the sum of weights, so this is not the weighted mean).⁠6 Other possible values are `"SUM"` and `"NONE"`.

- The `call()` method takes the labels and predictions, computes all the instance losses, and returns them.

* The `get_config()` method returns a dictionary mapping each hyperparameter name to its value. It first calls the parent class’s `get_config()` method, then adds the new hyperparameters to this dictionary.⁠7

A continuación, puede utilizar cualquier instancia de esta clase cuando compile el modelo:

In [ ]:
model.compile(loss=HuberLoss(2.), optimizer="nadam")

Cuando guarde el modelo, el umbral se guardará junto con él; y cuando cargue el modelo, solo tiene que asignar el nombre de la clase a la clase en sí:

In [ ]:
model = tf.keras.models.load_model("my_model_with_a_custom_loss_class",
                                   custom_objects={"HuberLoss": HuberLoss})

Cuando guarda un modelo, Keras llama al método `get_config()` de la instancia de pérdida y guarda la configuración en el formato SavedModel. Cuando carga el modelo, llama al método de clase `from_config()` en la clase `HuberLoss`: este método es implementado por la clase base (`Loss`) y crea una instancia de la clase, pasando `**config` al constructor.

¡Eso es todo por las pérdidas! Como verás ahora, las funciones de activación personalizadas, los inicializadores, los regularizadores y las restricciones no son muy diferentes.


### Funciones de activación personalizadas, inicializadores, regularizadores y restrinciones

La mayoría de las funcionalidades de Keras, como pérdidas, regularizadores, restricciones, inicializadores, métricas, funciones de activación, capas e incluso modelos completos, se pueden personalizar de la misma manera. La mayoría de las veces, sólo necesitarás escribir una función simple con las entradas y salidas apropiadas. A continuación se muestran ejemplos de una función de activación personalizada (equivalente a `tf.keras.activations.softplus()` o `tf.nn.softplus()`), un inicializador Glorot personalizado (equivalente a tf.keras.initializers.glorot_normal()), un Regularizador ℓ1 (equivalente a `tf.keras.regularizers.l1(0.01)`) y una restricción personalizada que garantiza que todos los pesos sean positivos (equivalente a `tf.keras.con⁠straints.nonneg()` o `tf.nn.relu()` ):

In [25]:
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):  # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

Como puede ver, los argumentos dependen del tipo de función personalizada. Estas funciones personalizadas se pueden utilizar normalmente, como se muestra aquí:

In [26]:
layer = tf.keras.layers.Dense(1, activation=my_softplus,
                              kernel_initializer=my_glorot_initializer,
                              kernel_regularizer=my_l1_regularizer,
                              kernel_constraint=my_positive_weights)

La función de activación se aplicará a la salida de esta capa Dense, y su resultado se pasará a la siguiente capa. Los pesos de la capa se inicializarán utilizando el valor devuelto por el inicializador. En cada paso de entrenamiento, los pesos se pasarán a la función de regularización para calcular la pérdida de regularización, que se añadirá a la pérdida principal para obtener la pérdida final utilizada para el entrenamiento. Finalmente, se llamará a la función de restricción después de cada paso de entrenamiento, y los pesos de la capa serán reemplazados por los pesos restringidos.

Si una función tiene hiperparámetros que deben guardarse junto con el modelo, entonces querrá crear una subclase de la clase apropiada, como `tf.keras.regu⁠larizers.Reg⁠⁠ularizer`, `tf.keras.constraints.Constraint `, `tf.keras.initializers.Ini⁠tializer` o `tf.keras.layers.Layer` (para cualquier capa, incluidas las funciones de activación). Al igual que lo hizo con la pérdida personalizada, aquí hay una clase simple para la regularización ℓ1 que guarda su hiperparámetro `factor` (esta vez no necesita llamar al constructor principal ni al método `get_config()`, ya que no están definidos por la clase padre):

In [27]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        return {"factor": self.factor}


Tenga en cuenta que debe implementar el método `call()` para pérdidas, capas (incluidas funciones de activación) y modelos, o el método `__call__()` para regularizadores, inicializadores y restricciones. Para las métricas, las cosas son un poco diferentes, como verás ahora.

### Métricas personalizadas

Las pérdidas y las métricas no son conceptualmente lo mismo: las pérdidas (por ejemplo, la entropía cruzada) se utilizan por descenso de gradiente para entrenar un modelo, por lo que deben ser diferenciables (al menos en los puntos donde se evalúan), y sus gradientes no deben ser cero en todas partes. Además, está bien si no son fácilmente interpretables por los humanos. Por el contrario, las métricas (por ejemplo, la precisión) se utilizan para evaluar un modelo: deben ser más fáciles de interpretar, y pueden ser indiferenciables o tener cero gradientes en todas partes.

Dicho esto, en la mayoría de los casos, definir una función métrica personalizada es exactamente lo mismo que definir una función de pérdida personalizada. De hecho, incluso podríamos usar la función de pérdida de Huber que creamos anteriormente como métrica;⁠ funcionaría bien (y la persistencia también funcionaría de la misma manera, en este caso solo guardando el nombre de la función, `"huber_fn"` , no el umbral):

In [ ]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

Para cada lote durante el entrenamiento, Keras calculará esta métrica y hará un seguimiento de su media desde el comienzo de la época. La mayoría de las veces, esto es exactamente lo que quieres. ¡Pero no siempre! Considere la precisión de un clasificador binario, por ejemplo. Como viste en el capítulo 3, la precisión es el número de verdaderos positivos dividido por el número de predicciones positivas (incluidos tanto los verdaderos positivos como los falsos positivos). Supongamos que el modelo hizo cinco predicciones positivas en el primer lote, cuatro de las cuales fueron correctas: eso es un 80 % de precisión. Entonces supongamos que el modelo hizo tres predicciones positivas en el segundo lote, pero todas eran incorrectas: eso es un 0 % de precisión para el segundo lote. Si solo calculas la media de estas dos precisiones, obtienes el 40 %. Pero espera un segundo, ¡esa no es la precisión del modelo sobre estos dos lotes! De hecho, hubo un total de cuatro verdaderos positivos (4 + 0) de ocho predicciones positivas (5 + 3), por lo que la precisión general es del 50 %, no del 40 %. Lo que necesitamos es un objeto que pueda realizar un seguimiento del número de verdaderos positivos y el número de falsos positivos y que pueda calcular la precisión basada en estos números cuando se solicite. Esto es precisamente lo que hace la clase `tf.keras.metrics.Precision`:

In [29]:
precision = tf.keras.metrics.Precision()

In [30]:
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [31]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

En este ejemplo, creamos un objeto `Precision`, luego lo usamos como una función, pasándole las etiquetas y predicciones para el primer lote, luego para el segundo lote (opcionalmente también puede pasar los pesos de la muestra, si lo desea). Utilizamos el mismo número de verdaderos y falsos positivos que en el ejemplo que acabamos de discutir. Después del primer lote, devuelve una precisión del 80 %; luego, después del segundo lote, devuelve el 50 % (que es la precisión general hasta ahora, no la precisión del segundo lote). Esto se llama métrica de transmisión (o métrica de estado), ya que se actualiza gradualmente, lote tras lote.

En cualquier momento, podemos llamar al método `result()` para obtener el valor actual de la métrica. También podemos ver sus variables (seguimiento del número de verdaderos y falsos positivos) usando el atributo de variables, y podemos restablecer estas variables usando el método `reset_states()`:

In [32]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [33]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [34]:
precision.reset_states()  # both variables get reset to 0.0

Si necesita definir su propia métrica de transmisión personalizada, cree una subclase de la clase `tf.keras.metrics.Metric`. Aquí hay un ejemplo básico que realiza un seguimiento de la pérdida total de Huber y el número de casos vistos hasta ahora. Cuando se le pide el resultado, devuelve la proporción, que es solo la pérdida media de Huber:

In [35]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)  # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

Vamos a revisar este código:⁠

+ El constructor utiliza el método `add_weight()` para crear las variables necesarias para realizar un seguimiento del estado de la métrica en varios lotes; en este caso, la suma de todas las pérdidas de Huber ("total") y el número de instancias vistas hasta el momento. (`count`). Podrías crear variables manualmente si lo prefieres. Keras rastrea cualquier `tf.Variable` que se establece como atributo (y, de manera más general, cualquier objeto "rastreable", como capas o modelos).

- El método `update_state()` se llama cuando usas una instancia de esta clase como función (como hicimos con el objeto `Precision`). Actualiza las variables, dadas las etiquetas y predicciones para un lote (y los pesos de la muestra, pero en este caso los ignoramos).

+ El método `result()` calcula y devuelve el resultado final, en este caso la métrica media de Huber en todas las instancias. Cuando se utiliza la métrica como función, primero se llama al método `update_state()`, luego se llama al método `result()` y se devuelve su salida.

- También implementamos el método `get_config()` para garantizar que el `threshold` se guarde junto con el modelo.

+ La implementación predeterminada del método `reset_states()` restablece todas las variables a 0.0 (pero puedes anularla si es necesario).

#### NOTA

Keras se encargará de la persistencia variable sin problemas; no se requiere ninguna acción.

#### ----------------------------------------------------------------------

Cuando define una métrica usando una función simple, Keras la llama automáticamente para cada lote y realiza un seguimiento de la media durante cada época, tal como lo hicimos manualmente. Entonces, el único beneficio de nuestra clase `HuberMetric` es que se guardará el `threshold`. Pero, por supuesto, algunas métricas, como la precisión, no pueden simplemente promediarse en lotes: en esos casos, no hay otra opción que implementar una métrica de transmisión.

Ahora que ha construido una métrica de transmisión, ¡construir una capa personalizada parecerá un paseo por el parque!


### Capas personalizadas

De vez en cuando es posible que desee construir una arquitectura que contenga una capa exótica para la que TensorFlow no proporciona una implementación predeterminada. O simplemente desea construir una arquitectura muy repetitiva, en la que un bloque particular de capas se repite muchas veces, y sería conveniente tratar cada bloque como una sola capa. Para estos casos, querrás crear una capa personalizada.

Hay algunas capas que no tienen pesos, como `tf.keras.layers.Flatten` o `tf.keras.layers.ReLU`. Si desea crear una capa personalizada sin pesos, la opción más sencilla es escribir una función y envolverla en una capa `tf.keras.layers.Lambda`. Por ejemplo, la siguiente capa aplicará la función exponencial a sus entradas:

In [36]:
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

Esta capa personalizada se puede utilizar como cualquier otra capa, utilizando la API secuencial, la API funcional o la API de subclasificación. También puedes usarlo como una función de activación, o puedes usar activation `activation=tf.exp`. La capa exponencial se utiliza a veces en la capa de salida de un modelo de regresión cuando los valores a predecir tienen escalas muy diferentes (por ejemplo, 0,001, 10, 1.000). De hecho, la función exponencial es una de las funciones de activación estándar en Keras, por lo que puedes usar `activation="exponential"`

Como puede adivinar, para construir una capa de estado personalizada (es decir, una capa con pesos), necesita crear una subclase de la clase thetf `tf.keras.layers.Layer`. Por ejemplo, la siguiente clase implementa una versión simplificada de la capa `Dense`:

In [37]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": tf.keras.activations.serialize(self.activation)}

Vamos a revisar este código:

+ El constructor toma todos los hiperparámetros como argumentos (en este ejemplo, `units` y `activation`) y, lo que es más importante, también toma un argumento `**kwargs`. Llama al constructor principal y le pasa los `kwargs`: esto se encarga de los argumentos estándar como `input_shape`, `trainable` y `name`. Luego guarda los hiperparámetros como atributos, convirtiendo el argumento de `activation` en la función de activación apropiada usando la función `tf.keras.activations.get()` (acepta funciones, cadenas estándar como `"relu"` o `"swish"`, o simplemente `None`).

- La función del método `build()` es crear las variables de la capa llamando al método `add_weight()` para cada peso. El método `build()` se llama la primera vez que se utiliza la capa. En ese punto, Keras conocerá la forma de las entradas de esta capa y la pasará al método `build()`,⁠ que suele ser necesario para crear algunos de los pesos. Por ejemplo, necesitamos saber el número de neuronas en la capa anterior para crear la matriz de pesos de conexión (es decir, el `"kernel"`): esto corresponde al tamaño de la última dimensión de las entradas. Al final del método `build()` (y solo al final), debes llamar al método `build()` del padre: esto le dice a Keras que la capa está construida (simplemente establece self.built = True).

+ El método `call()` realiza las operaciones deseadas. En este caso, calculamos la multiplicación matricial de las entradas X y el núcleo de la capa, sumamos el vector de polarización y aplicamos la función de activación al resultado, y esto nos da la salida de la capa.

- El método `get_config()` es como en las clases personalizadas anteriores. Tenga en cuenta que guardamos la configuración completa de la función de activación llamando a `tf.keras.activa⁠tions.serialize()`.

¡Ahora puedes usar una capa `MyDense` como cualquier otra capa!


#### NOTA

Keras infiere automáticamente la forma de salida, excepto cuando la capa es dinámica (como verás en breve). En este caso (raro), debe implementar el método `compute_output_shape()`, que debe devolver un objeto `TensorShape`.

#### ------------------------------------------------------------------------------


Para crear una capa con múltiples entradas (por ejemplo, `Concatenate`), el argumento del método `call()` debe ser una tupla que contenga todas las entradas. Para crear una capa con múltiples salidas, el método `call()` debe devolver la lista de salidas. Por ejemplo, la siguiente capa de juguete toma dos entradas y devuelve tres salidas:

In [38]:
class MyMultiLayer(tf.keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return X1 + X2, X1 * X2, X1 / X2

Esta capa ahora se puede usar como cualquier otra capa, pero, por supuesto, solo utilizando las API funcionales y de subclasificación, no la API secuencial (que solo acepta capas con una entrada y una salida).

Si su capa necesita tener un comportamiento diferente durante el entrenamiento y durante las pruebas (por ejemplo, si usa capas `Dropout` o `BatchNormalization`), entonces debe agregar un argumento de `training` al método `call()` y usar este argumento para decidir qué hacer. Por ejemplo, creemos una capa que agregue ruido gaussiano durante el entrenamiento (para regularización) pero no haga nada durante las pruebas (Keras tiene una capa que hace lo mismo, `tf.keras.layers.GaussianNoise`):

In [39]:
class MyGaussianNoise(tf.keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=False):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

¡Con eso, ahora puedes construir cualquier capa personalizada que necesites! Ahora echemos un vistazo a cómo crear modelos personalizados.

### Modelos personalizados

Ya analizamos la creación de clases de modelo personalizadas en el Capítulo 10, cuando analizamos la API de subclasificación.⁠ Es sencillo: subclasificar la clase `tf.keras.Model`, crear capas y variables en el constructor e implementar el método `call()` para hacer lo que quieras que haga el modelo. Por ejemplo, supongamos que queremos construir el modelo representado en la Figura 12-3.


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1203.png)

(_Figure 12-3. Custom model example: an arbitrary model with a custom ResidualBlock layer containing a skip connection_)

Las entradas pasan por una primera capa densa, luego a través de un bloque residual compuesto por dos capas densas y una operación de adición (como verá en el Capítulo 14, un bloque residual agrega sus entradas a sus salidas), luego a través de este mismo bloque residual tres veces más, luego a través de un segundo bloque residual, y el resultado final pasa a través de una capa de salida densa. No te preocupes si este modelo no tiene mucho sentido; es solo un ejemplo para ilustrar el hecho de que puedes construir fácilmente cualquier tipo de modelo que quieras, incluso uno que contenga bucles y omita conexiones. Para implementar este modelo, lo mejor es crear primero una capa de `ResidualBlock`, ya que vamos a crear un par de bloques idénticos (y es posible que queramos reutilizarlo en otro modelo):

In [40]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu",
                                             kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

Esta capa es un poco especial ya que contiene otras capas. Keras maneja esto de forma transparente: detecta automáticamente que el atributo `hidden` contiene objetos rastreables (capas en este caso), por lo que sus variables se agregan automáticamente a la lista de variables de esta capa. El resto de esta clase se explica por sí mismo. A continuación, usemos la API de subclases para definir el modelo en sí:

In [41]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation="relu",
                                             kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

Creamos las capas en el constructor y las usamos en el método `call()`. Luego, este modelo se puede utilizar como cualquier otro modelo (compilarlo, ajustarlo, evaluarlo y utilizarlo para hacer predicciones). Si también desea poder guardar el modelo usando el método `save()` y cargarlo usando la función `tf.keras.models.load_model()`, debe implementar el método `get_config()` (como hicimos antes) tanto en el Clase `ResidualBlock` y clase `ResidualRegressor`. Alternativamente, puede guardar y cargar los pesos usando los métodos `save_weights()` y `load_weights()`.

La clase `Model` es una subclase de la clase `Layer`, por lo que los modelos se pueden definir y utilizar exactamente como capas. Pero un modelo tiene algunas funcionalidades adicionales, incluidos, por supuesto, sus métodos `compile()`, `fit()`, `evaluate()` y `predict()` (y algunas variantes), además del método `get_layer()` (que puede devolver cualquiera de los datos del modelo). capas por nombre o por índice) y el método save() (y soporte para `tf.keras.models.load_model()` y `tf.keras.models.clone_model()`).

#### PROPINA

Si los modelos proporcionan más funcionalidad que las capas, ¿por qué no definir cada capa como un modelo? Bueno, técnicamente podrías, pero generalmente es más limpio distinguir los componentes internos de tu modelo (es decir, capas o bloques de capas reutilizables) del modelo en sí (es decir, el objeto que entrenarás). El primero debería subclasificar la clase `Layer`, mientras que el segundo debería subclasificar la clase `Model`.

#### -------------------------------------------------------------------------------------


Con eso, puede construir de forma natural y concisa casi cualquier modelo que encuentre en un documento, utilizando la API secuencial, la API funcional, la API de subclasificación o incluso una mezcla de estos. ¿Casi cualquier modelo? Sí, todavía hay algunas cosas que tenemos que ver: en primer lugar, cómo definir pérdidas o métricas basadas en los elementos internos del modelo y, en segundo lugar, cómo construir un bucle de entrenamiento personalizado.

### Pérdidas y métricas basadas en el modelo interno

Las pérdidas y métricas personalizadas que definimos anteriormente se basaron en las etiquetas y las predicciones (y, opcionalmente, en los pesos de la muestra). Habrá momentos en los que querrás definir las pérdidas basadas en otras partes de tu modelo, como los pesos o las activaciones de sus capas ocultas. Esto puede ser útil para fines de regularización o para monitorear algún aspecto interno de su modelo.

Para definir una pérdida personalizada basada en los aspectos internos del modelo, calculela en función de cualquier parte del modelo que desee y luego pase el resultado al método `add_loss()`. Por ejemplo, creemos un modelo MLP de regresión personalizado compuesto por una pila de cinco capas ocultas más una capa de salida. Este modelo personalizado también tendrá una salida auxiliar encima de la capa oculta superior. La pérdida asociada con esta salida auxiliar se llamará pérdida de reconstrucción (ver Capítulo 17): es la diferencia media cuadrática entre la reconstrucción y las entradas. Al agregar esta pérdida de reconstrucción a la pérdida principal, alentaremos al modelo a preservar la mayor cantidad de información posible a través de las capas ocultas, incluso información que no es directamente útil para la tarea de regresión en sí. En la práctica, esta pérdida a veces mejora la generalización (es una pérdida de regularización). También es posible agregar una métrica personalizada usando el método `add_metric()` del modelo. Aquí está el código para este modelo personalizado con una pérdida de reconstrucción personalizada y una métrica correspondiente:

In [42]:
class ReconstructingRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(30, activation="relu",
                                             kernel_initializer="he_normal")
                       for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dim)
        self.reconstruction_mean = tf.keras.metrics.Mean(
            name="reconstruction_error")

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = tf.keras.layers.Dense(n_inputs)

    def call(self, inputs, training=False):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)

Vamos a revisar este código:

- El constructor crea el DNN con cinco capas densas ocultas y una capa de salida densa. También creamos una métrica de transmisión `Mean` para realizar un seguimiento del error de reconstrucción durante el entrenamiento.

* El método `build()` crea una capa extra densa que se utilizará para reconstruir las entradas del modelo. Debe crearse aquí porque su número de unidades debe ser igual al número de entradas, y este número se desconoce antes de llamar al método `build()`.⁠

- El método `call()` procesa las entradas a través de las cinco capas ocultas y luego pasa el resultado a través de la capa de reconstrucción, que produce la reconstrucción.

* Luego, el método `call()` calcula la pérdida de reconstrucción (la diferencia cuadrática media entre la reconstrucción y las entradas) y la agrega a la lista de pérdidas del modelo usando el método `add_loss()`.⁠ Observe que reducimos la pérdida de reconstrucción en multiplicándolo por 0,05 (este es un hiperparámetro que puedes ajustar). Esto garantiza que las pérdidas por reconstrucción no dominen a las pérdidas principales.

- A continuación, solo durante el entrenamiento, el método `call()` actualiza la métrica de reconstrucción y la agrega al modelo para que pueda mostrarse. En realidad, este ejemplo de código se puede simplificar llamando a `self.add_metric(recon_loss)`: Keras rastreará automáticamente la media por usted.

* Finalmente, el método `call()` pasa la salida de las capas ocultas a la capa de salida y devuelve su salida.

Tanto la pérdida total como la pérdida de reconstrucción se reducirán durante el entrenamiento:

Época 1/5 363/363 [========] - 1s 820us/paso - pérdida: 0,7640 - error de reconstrucción: 1.2728 Época 2/5 363/363 [========] - 0s 809us/paso - pérdida: 0,4584 - error de reconstrucción: 0,6340 [...]


En la mayoría de los casos, todo lo que hemos discutido hasta ahora será suficiente para implementar cualquier modelo que desee construir, incluso con arquitecturas, pérdidas y métricas complejas. Sin embargo, para algunas arquitecturas, como las GAN (ver Capítulo 17), tendrá que personalizar el bucle de entrenamiento en sí. Antes de llegar allí, debemos ver cómo calcular los gradientes automáticamente en TensorFlow.

### Computación de gradientes usando Autodiff

Para entender cómo usar el autodiff (ver Capítulo 10 y Apéndice B) para calcular los gradientes automáticamente, consideremos una función de juguete simple:

In [43]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

Si sabes cálculo, puedes encontrar analíticamente que la derivada parcial de esta función con respecto a `w1` es `6 * w1 + 2 * w2`. 

También puedes encontrar que su derivada parcial con respecto a `w2` es `2 * w1`. Por ejemplo, en el punto `(w1, w2) = (5, 3)`, estas derivadas parciales son iguales a 36 y 10, respectivamente, por lo que el vector gradiente en este punto es (36, 10). 

Pero si se tratara de una red neuronal, la función sería mucho más compleja, normalmente con decenas de miles de parámetros, y encontrar las derivadas parciales analíticamente a mano sería una tarea prácticamente imposible. Una solución podría ser calcular una aproximación de cada derivada parcial midiendo cuánto cambia la salida de la función cuando modificas el parámetro correspondiente en una pequeña cantidad:

In [44]:
w1, w2 = 5, 3

In [45]:
eps = 1e-6

In [46]:
(f(w1 + eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [47]:
(f(w1, w2 + eps) - f(w1, w2)) / eps

10.000000003174137

¡Parece correcto! Esto funciona bastante bien y es fácil de implementar, pero es sólo una aproximación y, lo que es más importante, es necesario llamar a `f()` al menos una vez por parámetro (no dos veces, ya que podríamos calcular `f(w1, w2)` solo una vez). Tener que llamar a `f()` al menos una vez por parámetro hace que este enfoque sea intratable para redes neuronales grandes. Entonces, en su lugar, deberíamos usar autodiff en modo inverso. TensorFlow hace esto bastante simple:

In [48]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])

Primero definimos dos variables `w1` y `w2`, luego creamos un contexto `tf.GradientTape` que registrará automáticamente cada operación que involucre una variable, y finalmente le pedimos a esta cinta que calcule los gradientes del resultado `z` con respecto a ambas variables `[w1, w2]`. Echemos un vistazo a los gradientes que calculó TensorFlow:

In [49]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

¡Perfecto! El resultado no solo es preciso (la precisión solo está limitada por los errores de punto flotante), sino que el método `gradient()` solo realiza los cálculos registrados una vez (en orden inverso), sin importar cuántas variables haya, por lo que es increíblemente eficiente. ¡Es como magia!

#### CONSEJO

Para ahorrar memoria, coloque solo el mínimo estricto dentro del bloque `tf.GradientTape()`. Alternativamente, pausa la grabación creando un bloque `with tape.stop_recording()` dentro del bloque `tf.GradientTape()`.

#### ----------------------------------------------------------------------------------

La cinta se borra automáticamente inmediatamente después de llamar a su método gradient(), por lo que obtendrá una excepción si intenta llamar a `gradient()` dos veces:

In [51]:
with tf.GradientTape() as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)  # returns tensor 36.0
dz_dw2 = tape.gradient(z, w2)  # raises a RuntimeError!

RuntimeError: A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)

Si necesita llamar a `gradient()` más de una vez, debe hacer que la cinta sea persistente y eliminarla cada vez que termine para liberar recursos:⁠

In [52]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)  # returns tensor 36.0
dz_dw2 = tape.gradient(z, w2)  # returns tensor 10.0, works fine now!
del tape

De forma predeterminada, la cinta sólo rastreará operaciones que involucren variables, por lo que si intenta calcular el gradiente de `z` con respecto a algo que no sea una variable, el resultado será `None`:

In [53]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])  # returns [None, None]

Sin embargo, puede forzar la cinta para que observe los tensores que desee y registre cada operación que los involucre. Luego puedes calcular gradientes con respecto a estos tensores, como si fueran variables:

In [54]:
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])  # returns [tensor 36., tensor 10.]

Esto puede ser útil en algunos casos, como si quieres implementar una pérdida de regularización que penalice las activaciones que varían mucho cuando las entradas varían poco: la pérdida se basará en el gradiente de las activaciones con respecto a las entradas. Dado que las entradas no son variables, deberá indicarle a la cinta que las mire.

La mayoría de las veces se utiliza una cinta de gradiente para calcular los gradientes de un único valor (normalmente la pérdida) con respecto a un conjunto de valores (normalmente los parámetros del modelo). Aquí es donde brilla el autodiff en modo inverso, ya que sólo necesita hacer una pasada hacia adelante y una pasada hacia atrás para obtener todos los gradientes a la vez. Si intenta calcular los gradientes de un vector, por ejemplo, un vector que contiene múltiples pérdidas, TensorFlow calculará los gradientes de la suma del vector. Entonces, si alguna vez necesita obtener los gradientes individuales (por ejemplo, los gradientes de cada pérdida con respecto a los parámetros del modelo), debe llamar al método `jacobian()` de la cinta: realizará una diferenciación automática en modo inverso una vez por cada pérdida en el vector. (todo en paralelo por defecto). Incluso es posible calcular derivadas parciales de segundo orden (las hessianas, es decir, las derivadas parciales de las derivadas parciales), pero esto rara vez es necesario en la práctica (consulte la sección “Cálculo de gradientes usando Autodiff” del cuaderno de este capítulo para ver un ejemplo). ).

En algunos casos, es posible que desee evitar que los gradientes se propaguen hacia atrás a través de alguna parte de su red neuronal. Para hacer esto, debes usar la función `tf.stop_gradient()`. La función devuelve sus entradas durante el paso hacia adelante (como `tf.identity()`), pero no deja pasar los gradientes durante la propagación hacia atrás (actúa como una constante):

In [55]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1, w2)  # the forward pass is not affected by stop_gradient()

gradients = tape.gradient(z, [w1, w2])  # returns [tensor 30., None]

Finalmente, es posible que ocasionalmente te encuentres con algunos problemas numéricos al calcular gradientes. Por ejemplo, si calcula los gradientes de la función de raíz cuadrada en x = 10^(–50) el resultado será infinito. En realidad, la pendiente en ese punto no es infinita, pero es más de lo que los flotantes de 32 bits pueden soportar:

In [56]:
x = tf.Variable(1e-50)

with tf.GradientTape() as tape:
    z = tf.sqrt(x)
    
tape.gradient(z, [x])

[<tf.Tensor: shape=(), dtype=float32, numpy=inf>]

Para resolver esto, suele ser una buena idea agregar un valor pequeño a x (como 10^(–6)) al calcular su raíz cuadrada.

La función exponencial también es una fuente frecuente de dolores de cabeza, ya que crece extremadamente rápido. Por ejemplo, la forma en que se definió `my_softplus()` anteriormente no es numéricamente estable. Si calcula `my_softplus(100.0)`, obtendrá infinito en lugar del resultado correcto (aproximadamente 100). Pero es posible reescribir la función para hacerla numéricamente estable: la función softplus se define como log(1 + exp(z)), que también es igual a log(1 + exp(–|z|)) + max(z , 0) (ver el cuaderno para la prueba matemática) y la ventaja de esta segunda forma es que el término exponencial no puede explotar. Entonces, aquí hay una mejor implementación de la función `my_softplus()`:

In [57]:
def my_softplus(z):
    return tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)

En algunos casos raros, una función numéricamente estable aún puede tener gradientes numéricamente inestables. En tales casos, tendrás que decirle a TensorFlow qué ecuación usar para los gradientes, en lugar de permitirle usar autodiff. Para esto, debe usar el decorador `@tf.cus⁠tom_gradient` al definir la función y devolver tanto el resultado habitual de la función como una función que calcula los gradientes. Por ejemplo, actualicemos la función `my_softplus()` para que también devuelva una función de gradientes numéricamente estable:

In [58]:
@tf.custom_gradient
def my_softplus(z):
    def my_softplus_gradients(grads):  # grads = backprop'ed from upper layers
        return grads * (1 - 1 / (1 + tf.exp(z)))  # stable grads of softplus

    result = tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)
    return result, my_softplus_gradients

Si conoces cálculo diferencial (consulta el cuaderno tutorial sobre este tema), puedes encontrar que la derivada de log(1 + exp(z)) es exp(z) / (1 + exp(z)). Pero esta forma no es estable: para valores grandes de z, termina calculando infinito dividido por infinito, lo que devuelve NaN. Sin embargo, con un poco de manipulación algebraica, puedes demostrar que también es igual a 1 – 1 / (1 + exp(z)), que es estable. La función `my_softplus_gradients()` utiliza esta ecuación para calcular los gradientes. Tenga en cuenta que esta función recibirá como entrada los gradientes que se propagaron hacia atrás hasta el momento, hasta la función `my_softplus()`, y de acuerdo con la regla de la cadena debemos multiplicarlos con los gradientes de esta función.

Ahora, cuando calculamos los gradientes de la función `my_softplus()`, obtenemos el resultado adecuado, incluso para valores de entrada grandes.

¡Felicidades! Ahora puedes calcular los gradientes de cualquier función (siempre que sea diferenciable en el punto donde la calculas), incluso bloquear la retropropagación cuando sea necesario, ¡y escribir tus propias funciones de gradiente! Probablemente esto represente más flexibilidad de la que necesitará, incluso si crea sus propios circuitos de entrenamiento personalizados. Verás cómo hacerlo a continuación.


### Bucles de entrenamiento personalizados

En algunos casos, es posible que el método `fit()` no sea lo suficientemente flexible para lo que necesita hacer. Por ejemplo, el artículo Amplio y Profundo que analizamos en el Capítulo 10 utiliza dos optimizadores diferentes: uno para el camino ancho y el otro para el camino profundo. Dado que el método `fit()` solo utiliza un optimizador (el que especificamos al compilar el modelo), implementar este documento requiere escribir su propio bucle personalizado.

También es posible que desee escribir ciclos de entrenamiento personalizados simplemente para sentirse más seguro de que hacen precisamente lo que usted pretende que hagan (tal vez no esté seguro de algunos detalles del método `fit()`). A veces puede parecer más seguro hacer todo explícito. Sin embargo, recuerde que escribir un ciclo de entrenamiento personalizado hará que su código sea más largo, más propenso a errores y más difícil de mantener.

#### TIP

A menos que esté aprendiendo o realmente necesite flexibilidad adicional, debería preferir usar el método `fit()` en lugar de implementar su propio ciclo de entrenamiento, especialmente si trabaja en equipo.

#### -------------------------------------------------------------------------------------

Primero, construyamos un modelo simple. No es necesario compilarlo, ya que manejaremos el ciclo de entrenamiento manualmente:

In [59]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          kernel_regularizer=l2_reg),
    tf.keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

A continuación, creemos una pequeña función que muestreará aleatoriamente un lote de instancias del conjunto de entrenamiento (en el Capítulo 13 analizaremos la API tf.data, que ofrece una alternativa mucho mejor):

In [60]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

También definamos una función que mostrará el estado del entrenamiento, incluido el número de pasos, el número total de pasos, la pérdida media desde el inicio de la época (usaremos la métrica `Mean` para calcularla) y otras métricas:

In [61]:
def print_status_bar(step, total, loss, metrics=None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}"
                          for m in [loss] + (metrics or [])])
    end = "" if step < total else "\n"
    print(f"\r{step}/{total} - " + metrics, end=end)

Este código se explica por sí mismo, a menos que no esté familiarizado con el formato de cadenas de Python: `{m.result():.4f}` formateará el resultado de la métrica como un flotante con cuatro dígitos después del punto decimal y utilizará `\r` (retorno de carro). junto con `end=""` garantiza que la barra de estado siempre se imprima en la misma línea.

¡Con eso, ponponmos a la obra! En primer lugar, tenemos que definir algunos hiperparámetros y elegir el optimizador, la función de pérdida y las métricas (solo el MAE en este ejemplo):

In [ ]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean(name="mean_loss")
metrics = [tf.keras.metrics.MeanAbsoluteError()]

¡Y ahora estamos listos para construir el bucle personalizado!

In [ ]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)

        print_status_bar(step, n_steps, mean_loss, metrics)

    for metric in [mean_loss] + metrics:
        metric.reset_states()

Están pasando muchas cosas en este código, así que vamos a revisarlo:

- Creamos dos bucles anidados: uno para las épocas, el otro para los lotes dentro de una época.

* Luego muestreamos un lote aleatorio del conjunto de entrenamiento.

- Dentro del bloque `tf.GradientTape()`, hacemos una predicción para un lote, usando el modelo como función, y calculamos la pérdida: es igual a la pérdida principal más las otras pérdidas (en este modelo, hay una pérdida de regularización por capa). Dado que la función `mean_squared_error()` devuelve una pérdida por instancia, calculamos la media sobre el lote usando `tf.reduce_mean()` (si quisieras aplicar pesos diferentes a cada instancia, aquí es donde lo harías). Las pérdidas de regularización ya se han reducido a un solo escalar cada una, por lo que solo necesitamos sumarlas (usando `tf.add_n()`, que suma múltiples tensores de la misma forma y tipo de datos).

* A continuación, le pedimos a la cinta que calcule los gradientes de la pérdida con respecto a cada variable entrenable, ¡no todas las variables!, y los aplicamos al optimizador para realizar un paso de descenso de gradiente.

- Luego actualizamos la pérdida media y las métricas (sobre la época actual), y mostramos la barra de estado.

* Al final de cada época, restablecemos los estados de la pérdida media y las métricas.


Si desea aplicar recorte de degradado (consulte el Capítulo 11), configure el hiperparámetro `clipnorm` o `clipvalue` del optimizador. Si desea aplicar cualquier otra transformación a los gradientes, simplemente hágalo antes de llamar al método `apply_gradients()`. Y si desea agregar restricciones de peso a su modelo (por ejemplo, configurando `kernel_constraint` o `bias_constraint` al crear una capa), debe actualizar el bucle de entrenamiento para aplicar estas restricciones justo después de `apply_gradients()`, así:

In [ ]:
for variable in model.variables:
    if variable.constraint is not None:
        variable.assign(variable.constraint(variable))

#### ADVERTENCIA

No olvide establecer `training=True` cuando llame al modelo en el ciclo de entrenamiento, especialmente si su modelo se comporta de manera diferente durante el entrenamiento y las pruebas (por ejemplo, si usa `BatchNormalization` o `Dropout`). Si es un modelo personalizado, asegúrese de propagar el argumento `"training"` a las capas a las que llama su modelo.

Como puedes ver, hay bastantes cosas que necesitas hacer bien, y es fácil cometer un error. Pero en el lado positivo, tienes el control total.

Ahora que sabe cómo personalizar cualquier parte de sus modelos⁠ y algoritmos de entrenamiento, veamos cómo puede usar la función de generación automática de gráficos de TensorFlow: puede acelerar considerablemente su código personalizado, y también lo hará portátil a cualquier plataforma compatible con TensorFlow (consulte el capítulo 19).


## Funciones y gráficos de TensorFlow

De vuelta en TensorFlow 1, los gráficos eran inevitables (al isí como las complejidades que venían con ellos) porque eran una parte central de la API de TensorFlow. Desde TensorFlow 2 (lanzado en 2019), los gráficos siguen ahí, pero no tan centrales, y son mucho (¡mucho!) más fácil de usar. Para mostrar lo simple que es, comencemos con una función trivial que calcula el cubo de su entrada:

In [63]:
def cube(x):
    return x ** 3

Obviamente, podemos llamar a esta función con un valor de Python, como un int o un float, o podemos llamarla con un tensor:

In [64]:
cube(2)

8

In [65]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

Ahora, vamos a usar `tf.function()` para convertir esta función de Python a una función de Tensorflow.

In [66]:
tf_cube = tf.function(cube)
tf_cube

Esta función TF se puede utilizar exactamente igual que la función original de Python, y devolverá el mismo resultado (pero siempre como tensores):

In [67]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [68]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

¡Debajo del capó, `tf.function()` analizó los cálculos realizados por la función `cube()` y generó un gráfico de cálculo equivalente! Como puede ver, fue bastante indoloro (veremos cómo funciona esto en breve). Alternativamente, podríamos haber usado `tf.function` como decorador; En realidad, esto es más común:

In [69]:
@tf.function
def tf_cube(x):
    return x ** 3

La función original de Python sigue disponible via función TF atributo `python_function`, en caso de que alguna vez lo necesites.

In [70]:
tf_cube.python_function(2)

8

TensorFlow optimiza el gráfico de cálculo, podando los nodos no utilizados, simplificando las expresiones (por ejemplo, 1 + 2 sería reemplazado por 3) y más. Una vez que el gráfico optimizado está listo, la función TF ejecuta de manera eficiente las operaciones en el gráfico, en el orden apropiado (y en paralelo cuando puede). Como resultado, una función TF generalmente se ejecutará mucho más rápido que la función Python original, especialmente si realiza cálculos complejos.⁠ La mayoría de las veces no necesitarás saber más que eso: cuando quieras impulsar una función de Python, simplemente transfórmala en una función TF. ¡Eso es todo!

Además, si configura `jit_compile=True` al llamar a `tf.function()`, TensorFlow utilizará álgebra lineal acelerada (XLA) para compilar núcleos dedicados para su gráfico, a menudo fusionando múltiples operaciones. Por ejemplo, si su función TF llama a `tf.reduce_sum(a * b + c)`, entonces, sin XLA, la función primero necesitaría calcular `a * b` y almacenar el resultado en una variable temporal, luego agregar c a esa variable y, por último, llame a `tf.reduce_sum()` en el resultado. Con XLA, todo el cálculo se compila en un único núcleo, que calculará `tf.reduce_sum(a * b + c)` de una sola vez, sin utilizar ninguna variable temporal grande. Esto no sólo será mucho más rápido, sino que también utilizará muchísimo menos RAM.

Cuando escribe una función de pérdida personalizada, una métrica personalizada, una capa personalizada o cualquier otra función personalizada y la usa en un modelo de Keras (como lo hemos hecho a lo largo de este capítulo), Keras convierte automáticamente su función en una función TF. no es necesario utilizar `tf.function()`. Así que la mayoría de las veces, la magia es 100% transparente. Y si desea que Keras use XLA, solo necesita configurar `jit_compile=True` al llamar al método `compile()`. 

¡Fácil!

#### TIP

Puede indicarle a Keras que no convierta sus funciones de Python en funciones TF configurando `dynamic=True` al crear una capa personalizada o un modelo personalizado. Alternativamente, puede configurar `run_eagerly=True` al llamar al método `compile()` del modelo.

#### -------------------------------------------------------------------------------------

De forma predeterminada, una función TF genera un nuevo gráfico para cada conjunto único de formas de entrada y tipos de datos y lo almacena en caché para llamadas posteriores. Por ejemplo, si llama a `tf_cube(tf.constant(10))`, se generará un gráfico para tensores int32 de forma []. Luego, si llama a `tf_cube(tf.constant(20))`, se reutilizará el mismo gráfico. Pero si luego llamas a `tf_cube(tf.constant([10, 20]))`, se generará un nuevo gráfico para tensores de forma int32 [2]. Así es como las funciones TF manejan el polimorfismo (es decir, diferentes tipos y formas de argumentos). Sin embargo, esto solo es cierto para los argumentos tensoriales: si pasa valores numéricos de Python a una función TF, se generará un nuevo gráfico para cada valor distinto: por ejemplo, llamar a `tf_cube(10)` y `tf_cube(20)` generará dos gráficos.


#### ADVERTENCIA

Si llama a una función TF muchas veces con diferentes valores numéricos de Python, se generarán muchos gráficos, ralentizando su programa y utilizando mucha RAM (debe eliminar la función TF para liberarla). Los valores de Python deben reservarse para argumentos que tengan pocos valores únicos, como hiperparámetros como el número de neuronas por capa. Esto permite a TensorFlow optimizar mejor cada variante de su modelo.

#### -------------------------------------------------------------------------------------


### Autográfico y rastreo

Entonces, ¿cómo genera gráficos TensorFlow? Comienza analizando el código fuente de la función Python para capturar todas las declaraciones de flujo de control, como bucles `for`, bucles `while` y declaraciones `if`, así como declaraciones `break`, `continue` y `return`. 

Este primer paso se llama _AutoGraph_. La razón por la que TensorFlow tiene que analizar el código fuente es que Python no proporciona ninguna otra forma de capturar declaraciones de flujo de control: ofrece métodos mágicos como `__add__()` y `__mul__()` para capturar operadores como `+` y `*`, pero no hay `__ while__( )` o `__if__()` métodos mágicos. 

Después de analizar el código de la función, AutoGraph genera una versión mejorada de esa función en la que todas las declaraciones de flujo de control se reemplazan por las operaciones apropiadas de TensorFlow, como `tf. while_loop()` para bucles y `tf.cond()` para declaraciones `if`. 

Por ejemplo, en la Figura 12-4, AutoGraph analiza el código fuente de la función Python `sum_squares()` y genera la función `tf__sum_squares()`. 

En esta función, el bucle for se reemplaza por la definición de la función `loop_body()` (que contiene el cuerpo del bucle for original), seguida de una llamada a la función `for_stmt()`. Esta llamada creará la operación `tf.while_loop()` apropiada en el gráfico de cálculo.


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1204.png)

(_Figura 12-4. Cómo TensorFlow genera gráficos usando AutoGraph y rastreo_)

A continuación, TensorFlow llama a esta función "actualizada", pero en lugar de pasar el argumento, pasa un tensor simbólico: un tensor sin ningún valor real, solo un nombre, un tipo de datos y una forma. Por ejemplo, si llama a `sum_squares(tf.constant(10))`, entonces la función `tf__sum_squares()` se llamará con un tensor simbólico de tipo int32 y forma []. La función se ejecutará en modo gráfico, lo que significa que cada operación de TensorFlow agregará un nodo en el gráfico para representarse a sí misma y a sus tensores de salida (a diferencia del modo normal, llamado ejecución ansiosa o modo ansioso). En el modo gráfico, las operaciones TF no realizan ningún cálculo. El modo gráfico era el modo predeterminado en TensorFlow 1. En la Figura 12-4, puede ver cómo se llama a la función `tf__sum_squares()` con un tensor simbólico como argumento (en este caso, un tensor int32 de forma []) y el gráfico final. generado durante el rastreo. Los nodos representan operaciones y las flechas representan tensores (tanto la función generada como el gráfico están simplificados).

#### PROPINA

Para ver el código fuente de la función generada, puede llamar atf `tf.autograph.to_code(sum_squares.python_function)` El código no está destinado a ser bonito, pero a veces puede ayudar con la depuración.

#### ----------------------------------------------------------------------------

## Reglas de la función TF

La mayoría de las veces, convertir una función de Python que realiza operaciones de TensorFlow en una función de TF es trivial: decórala con `@tf.function` o deja que Keras se encargue de ello por ti. Sin embargo, hay algunas reglas a respetar:

* Si llama a cualquier biblioteca externa, incluida NumPy o incluso la biblioteca estándar, esta llamada se ejecutará solo durante el seguimiento; no será parte del gráfico. De hecho, un gráfico de TensorFlow solo puede incluir construcciones de TensorFlow (tensores, operaciones, variables, conjuntos de datos, etc.). Por lo tanto, asegúrese de usar `tf.reduce_sum()` en lugar de `np.sum()`, `tf.sort()` en lugar de la función incorporada `sorted()`, etc. (a menos que realmente desee que el código se ejecute solo durante el seguimiento). ). Esto tiene algunas implicaciones adicionales:

* * Si define una función TF `f(x)` que simplemente devuelve `np.random.rand()`, solo se generará un número aleatorio cuando se rastree la función, por lo que `f(tf.constant(2.)) ` y `f(tf.constant(3.))` devolverán el mismo número aleatorio, pero `f(tf.constant([2., 3.]))` devolverá uno diferente. Si reemplaza `np.random.rand()` con `tf.random.uniform([])`, se generará un nuevo número aleatorio en cada llamada, ya que la operación será parte del gráfico.

* * Si su código que no es TensorFlow tiene efectos secundarios (como registrar algo o actualizar un contador de Python), entonces no debe esperar que esos efectos secundarios ocurran cada vez que llame a la función TF, ya que solo ocurrirán cuando se rastree la función.

* * Puede envolver código de Python arbitrario en la operación atf `tf.py_function()`, pero hacerlo obstaculizará el rendimiento, ya que TensorFlow no podrá hacer ninguna optimización gráfica en este código. También reducirá la portabilidad, ya que el gráfico solo se ejecutará en plataformas donde Python esté disponible (y donde estén instaladas las bibliotecas correctas).

- Puedes llamar a otras funciones de Python o TF, pero deben seguir las mismas reglas, ya que TensorFlow capturará sus operaciones en el gráfico de computación. Tenga en cuenta que estas otras funciones no necesitan estar decoradas con `@tf.function`.

* Si la función crea una variable de TensorFlow (o cualquier otro objeto de TensorFlow con estado, como un conjunto de datos o una cola), debe hacerlo en la primera llamada, y solo entonces, o de lo contrario obtendrá una excepción. Generalmente es preferible crear variables fuera de la función TF (por ejemplo, en el método `build()` de una capa personalizada). Si desea asignar un nuevo valor a la variable, asegúrese de llamar a su método `assign()` en lugar de usar el operador `=`.

- El código fuente de su función Python debería estar disponible para TensorFlow. Si el código fuente no está disponible (por ejemplo, si define su función en el shell de Python, que no da acceso al código fuente, o si implementa solo los archivos de Python *.pyc compilados en la producción), entonces el proceso de generación de gráficos fallará o tendrá una funcionalidad limitada.

* TensorFlow solo capturará bucles for que iteren sobre un tensor o un `tf.data.Dataset` (consulte el Capítulo 13). Por lo tanto, asegúrese de utilizar `for i in tf.range(x)` en lugar de `for i in range(x)`; de lo contrario, el bucle no se capturará en el gráfico. En cambio, se ejecutará durante el seguimiento. (Esto puede ser lo que desea si el bucle for está destinado a construir el gráfico; por ejemplo, para crear cada capa en una red neuronal).albert


- Como siempre, por razones de rendimiento, deberías preferir una implementación vectorial siempre que puedas, en lugar de usar bucles.

¡Es hora de resumir! En este capítulo comenzamos con una breve descripción de TensorFlow, luego ansalzamos la API de bajo nivel de TensorFlow, que incluye tensores, operaciones, variables y estructuras de datos especiales. Luego utilizamos estas herramientas para personalizar casi todos los componentes de la API de Keras. Finalmente, anotamos cómo las funciones TF pueden aumentar el rendimiento, cómo se generan los gráficos utilizando AutoGraph y el rastreo, y qué reglas seguir cuando escribe funciones TF (si desea abrir la caja negra un poco más y explorar los gráficos generados, encontrará detalles técnicos en el Apéndice D).

En el próximo capítulo, veremos cómo cargar y preprocesar datos de manera eficiente con TensorFlow.

# Ejercicios

- ¿Cómo describirías TensorFlow en una frase corta? ¿Cuáles son sus características principales? ¿Puedes nombrar otras bibliotecas populares de aprendizaje profundo?

- ¿Es TensorFlow un reemplazo directo para NumPy? ¿Cuáles son las principales diferencias entre los dos?

- Do you get the same result with tf.range(10) and tf.constant(np.​ara⁠nge(10))?

- ¿Puedes nombrar otras seis estructuras de datos disponibles en TensorFlow, más allá de los tensores regulares?
- Puede definir una función de pérdida personalizada escribiendo una función o subclasificando la clase tf.keras.losses.Loss. ¿Cuándo usarías cada opción?

- Del mismo modo, puede definir una métrica personalizada en una función o como una subclase de tf.keras.metrics.Metric. ¿Cuándo usarías cada opción?

- ¿Cuándo deberías crear una capa personalizada frente a un modelo personalizado?

- ¿Cuáles son algunos casos de uso que requieren escribir su propio bucle de entrenamiento personalizado?

- ¿Pueden los componentes personalizados de Keras contener código Python arbitrario o deben ser convertibles a funciones TF?

- ¿Cuáles son las principales reglas que hay que respetar si quieres que una función sea convertible en una función TF?

- ¿Cuándo necesitarías crear un modelo dinámico de Keras? ¿Cómo lo haces? ¿Por qué no haces que todos tus modelos sean dinámicos?

- Implemente una capa personalizada que realice la normalización de la capa (utilizaremos este tipo de capa en el capítulo 15):

- - The build() method should define two trainable weights α and β, both of shape input_shape[-1:] and data type tf.float32. α should be initialized with 1s, and β with 0s.

- - The call() method should compute the mean μ and standard deviation σ of each instance’s features. For this, you can use tf.nn.moments(inputs, axes=-1, keepdims=True), which returns the mean μ and the variance σ2 of all instances (compute the square root of the variance to get the standard deviation). Then the function should compute and return α ⊗ (X – μ)/(σ + ε) + β, where ⊗ represents itemwise multiplication (*) and ε is a smoothing term (a small constant to avoid division by zero, e.g., 0.001).

- - Asegúrese de que su capa personalizada produzca la misma (o casi la misma) salida que la capa thetftf.keras.layers.LayerNormalization.

- Entrena a un modelo usando un bucle de entrenamiento personalizado para abordar el conjunto de datos de Fashion MNIST (ver Capítulo 10):

- - Muestra la época, la iteración, la pérdida media de entrenamiento y la precisión media en cada época (actualizada en cada iteración), así como la pérdida de validación y la precisión al final de cada época.

- - Intente usar un optimizador diferente con una tasa de aprendizaje diferente para las capas superiores y las capas inferiores.
Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.

1 Sin embargo, la biblioteca PyTorch de Facebook es actualmente más popular en el mundo académico: más artículos citan a PyTorch que TensorFlow o Keras. Además, la biblioteca JAX de Google está ganando impulso, especialmente en el mundo académico.

2 TensorFlow incluye otra API de aprendizaje profundo llamada API de estimadores, pero ahora está obsoleta.

3 Si alguna vez lo necesitas (pero probablemente no lo harás), puedes escribir tus propias operaciones usando la API de C++.

4 Para obtener más información sobre los TPU y cómo funcionan, visite https://homl.info/tpus.

5 Una excepción notable es tf.math.log(), que se usa comúnmente pero no tiene un alias tf.log(), ya que podría confundirse con el registro.

6 No sería una buena idea usar un medio ponderado: si lo hiciera, entonces dos casos con el mismo peso pero en lotes diferentes tendrían un impacto diferente en el entrenamiento, dependiendo del peso total de cada lote.

7 The {**x, [...]} syntax was added in Python 3.5, to merge all the key/value pairs from dictionary x into another dictionary. Since Python 3.9, you can use the nicer x | y syntax instead (where x and y are two dictionaries).

8 Sin embargo, la pérdida de Huber rara vez se utiliza como métrica; generalmente se prefiere el MAE o MSE.

9 Esta clase es solo para fines ilustrativos. Una implementación más simple y mejor solo subclasificaría la clase tf.keras.metrics.Mean; consulte la sección "Métricas de transmisión" del cuaderno de este capítulo para ver un ejemplo.

10 La API de Keras llama a este argumento input_shape, pero como también incluye la dimensión del lote, prefiero llamarlo batch_input_shape.

11 El nombre "API de subclasificación" en Keras generalmente se refiere solo a la creación de modelos personalizados mediante subclasificación, aunque muchas otras cosas se pueden crear mediante subclasificación, como has visto en este capítulo.

12 Due to TensorFlow issue #46858, the call to super().build() may fail in this case, unless the issue was fixed by the time you read this. If not, you need to replace this line with self.built = True.

13 You can also call add_loss() on any layer inside the model, as the model recursively gathers losses from all of its layers.

14 Si la cinta está fuera de alcance, por ejemplo, cuando la función que la usó regresa, el recolector de basura de Python la eliminará por usted.

15 Con la excepción de los optimizadores, ya que muy pocas personas los personalizan; consulte la sección "Optimizadores personalizados" en el cuaderno para ver un ejemplo.

16 However, in this trivial example, the computation graph is so small that there is nothing at all to optimize, so tf_cube() actually runs much slower than cube().